In [1]:
import os
import xarray as xr
import numpy as np

import sys
sys.path.append("..")
import utils

In [ ]:
experiments = ["historical", "ssp126", "ssp585"]

fold = f"../../data/CMIP6/"
models = sorted(utils.dirtodict(fold))[1:]


def mean_it_up(model, variant, experiment):
    ofo = f"../../results/SST/CMIP6/{model}/{variant}"

    if not os.path.exists(ofo):
        os.makedirs(ofo)
    path = f"{ofo}/mean_sst_{experiment}.nc"
    if os.path.exists(path):
        return

    else:
        ds = utils.open_and_process_dataset(
            f"../../data/CMIP6/{model}/{variant}/tos/{experiment}/*.nc"
        )
        _, _, lat_idx, lon_idx = utils.get_1d_lat_lon(ds)
        ds = ds.mean([lat_idx, lon_idx])
        ds = ds.groupby("time.year").mean(dim="time")
        ds.to_netcdf(path)
    return

for model in models:
    if model != 'NESM3':
        print('\n' + model,':', end=' ')
        for variant in sorted(utils.dirtodict(f"../../data/CMIP6/{model}/").keys())[1:]:
            print(variant, end=', ')
            for experiment in experiments:
                mean_it_up(model, variant, experiment)